In [1]:
# Notebook 01 - Coleta de Tweets com Tweepy e Bearer Token

# Este notebook realiza a coleta de tweets relacionados a modelos de automóveis usando **Tweepy** e o **Bearer Token** da API do Twitter (X).

In [2]:
import tweepy
import pandas as pd
from datetime import datetime
import os # Importe a biblioteca os para manipulação de diretório
import time  # Importa a biblioteca 'time'

print('Bibliotecas carregadas.')

Bibliotecas carregadas.


In [3]:
# Substitua com o seu Bearer Token (mantenha-o seguro!)
# É recomendado usar variáveis de ambiente para não expor seu token.
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAN9x4AEAAAAAD2SskmOwVjc0OLx5lHQlg9m2EuI%3DIu9L3E8cnr8LxMbaBqx5nICrMEUJ9aDhAkxd0AdXdZcppb0x8C' # <<-- SUBSTITUA PELO SEU BEARER TOKEN REAL

# Autenticação com o Tweepy usando Bearer Token
# O Client é a forma recomendada para interagir com a API v2 do Twitter
client = tweepy.Client(BEARER_TOKEN)

print('Autenticação com Bearer Token realizada.')

Autenticação com Bearer Token realizada.


In [4]:
# Lista de modelos de carro
modelos = ['Onix', 'HB20']
limite_total_tweets = 20 # <-- Variável definida aqui

In [7]:
# Função para coletar tweets com Paginator (API V2)
def coletar_tweets_com_paginator(query, limite_total=10):
    tweets = []

    # Cria um objeto Paginator para a busca de tweets recentes
    # max_results por requisição é 100 para a API V2 (gratuita)
    paginator = tweepy.Paginator(
        client.search_recent_tweets,
        query=f'{query} lang:pt', 
        max_results=10, 
        tweet_fields=["created_at", "author_id", "text", "public_metrics"] # Campos essenciais
        # Você pode adicionar mais campos aqui se precisar, como 'public_metrics', 'entities', etc.
    )

    # Itera sobre os tweets retornados pelo paginator, limitado pelo limite_total
    # O flatten() ajuda a obter diretamente os objetos Tweet
    for tweet in paginator.flatten(limit=limite_total):
        tweets.append([tweet.created_at, tweet.author_id, tweet.text])

    df = pd.DataFrame(tweets, columns=['date', 'user_id', 'content'])
    return df


In [8]:
# Diretório para salvar os dados
# Cria o diretório de dados se ele não existir
data_dir = '../data/raw'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

dfs = {}
for i, modelo in enumerate(modelos):
    print(f'Coletando tweets para: "{modelo}"...')
    
    try:
        dfs[modelo] = coletar_tweets_com_paginator(modelo, limite_total_tweets)
        
        if not dfs[modelo].empty:
            caminho_arquivo = os.path.join(data_dir, f'{modelo}_tweets.csv')
            dfs[modelo].to_csv(caminho_arquivo, index=False)
            print(f'Tweets de "{modelo}" salvos em {caminho_arquivo} ({len(dfs[modelo])} tweets).')
    except tweepy.TooManyRequests as e:
        print(f"Erro de 'Too Many Requests' para o modelo {modelo}. Limite da API atingido.")
        print("Tente novamente em 15 minutos. A coleta será interrompida.")
        break  # Interrompe o loop
    
    # Adiciona um atraso entre as requisições para evitar o limite de taxa.
    # O Paginator faz a paginação, mas um atraso entre cada modelo é crucial.
    if i < len(modelos) - 1:
        print("Aguardando 15 minutos para resetar o limite da API...")
        time.sleep(900)  # 900 segundos = 15 minutos
        
print('\nProcesso de coleta de tweets finalizado.')

Coletando tweets para: "Onix"...
Erro de 'Too Many Requests' para o modelo Onix. Limite da API atingido.
Tente novamente em 15 minutos. A coleta será interrompida.

Processo de coleta de tweets finalizado.
